In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pakistan-toshakhana-files/Refined_TK_data ver 2.csv
/kaggle/input/pakistan-toshakhana-files/Toshakhana Files 2.xlsx


In [2]:
tosha_df=pd.read_csv("/kaggle/input/pakistan-toshakhana-files/Refined_TK_data ver 2.csv")
tosha_df.head()

,Detail of Gifts,Item Category,Name of Recipient,Affiliation,Date,Assessed Value,Retention Cost,Retained,Remarks
0,Two small silver\nBowls,Decoration Pieces,"Mr. Abdul Sattar, Minister for Foreign\nA...",Gen. Musharrafarraf,1/8/2002,5000.0,0.0,Yes,Yes
1,One Pilot Pen with\nink,Pen,"Mr. Abdul Sattar, Minister for Foreign\nA...",Gen. Musharraf,1/8/2002,3500.0,0.0,Yes,Yes
2,One silver vase,Decoration Pieces,"Mr. Shaukat Aziz, Minister for Finance",Gen. Musharraf,1/9/2002,7500.0,0.0,Yes,Yes
3,One (decoration)\nfolding Screen,Decoration Pieces,"Mr. Abdul Sattar, Minister for Foreign\nAf...",Gen. Musharraf,1/15/2002,2200.0,0.0,Yes,Yes
4,One Decoration\nPiece,Decoration Pieces,"Mr. Abdul Sattar, Minister for Foreign\nA...",Gen. Musharraf,1/22/2002,2000.0,0.0,Yes,Yes


In [3]:
print(tosha_df.describe())
print(tosha_df.count())

       Assessed Value  Retention Cost
count    4.213000e+03    4.205000e+03
mean     3.210531e+05    4.851276e+04
std      3.894070e+06    6.184538e+05
min      0.000000e+00    0.000000e+00
25%      1.700000e+03    0.000000e+00
50%      8.000000e+03    0.000000e+00
75%      4.000000e+04    2.250000e+03
max      1.400000e+08    2.017800e+07
Detail of Gifts      4208
Item Category        2338
Name of Recipient    4214
Affiliation          3207
Date                 4211
Assessed Value       4213
Retention Cost       4205
Retained             4203
Remarks              4203
dtype: int64


Droping rows that have null values for the most important features 

In [4]:
tosha_df.isnull().sum()

Detail of Gifts         6
Item Category        1876
Name of Recipient       0
Affiliation          1007
Date                    3
Assessed Value          1
Retention Cost          9
Retained               11
Remarks                11
dtype: int64

In [5]:
tosha_df=tosha_df.dropna(subset=["Detail of Gifts","Assessed Value","Retained"])
tosha_df["Retention Cost"]=tosha_df["Retention Cost"].fillna(0.0)
#tosha_df.isnull().sum()

Replacing missing date with ffill

In [6]:
tosha_df["Date"].replace("NaN",np.nan,inplace=True)

#print(tosha_df.isnull().sum())
#print(tosha_df.count())

In [7]:
tosha_df.loc[tosha_df["Date"].isnull(),"Date"]=tosha_df["Date"].ffill()
#tosha_df.isnull().sum()

Fill null values of affiliation with 'Other/s'

In [8]:
tosha_df["Affiliation"].fillna('Other/s',inplace=True)
tosha_df.reset_index(drop=True,inplace=True)
#tosha_df.head(10)

Spliting the Name of the recipient and the office that person holds into two columns

In [9]:
tosha_df[["Recipient","Office"]]=tosha_df['Name of Recipient'].str.split(',',n=1,expand=True)
tosha_df.drop('Name of Recipient',axis=1,inplace=True)
#tosha_df.head(10)

In [10]:
nulldf=tosha_df[tosha_df['Office'].isnull()].head(10)
nulldf.head(10)

,Detail of Gifts,Item Category,Affiliation,Date,Assessed Value,Retention Cost,Retained,Remarks,Recipient,Office
22,One metallic\nvase,Decoration Pieces,Gen. Musharraf,2/19/2002,850.0,0.0,Yes,Yes,Minister for Petroleum & Natural\nResources,None
23,One cufflinks\nInsignia on it,Jewellery/Accessories,Gen. Musharraf,2/19/2002,0.0,0.0,Yes,Yes,Minister for Petroleum & Natural\nResources,None
47,One small\nwooden box with\nsix packets of\...,Food,Gen. Musharraf,3/22/2002,0.0,0.0,Yes,Yes,Minister for Information & Media\nDevelopment,None
49,One tea set,Crockery,Gen. Musharraf,3/27/2002,4000.0,0.0,Yes,Yes,Foreign Secretary,None
58,One Cassette\nRecorder,Tech,Gen. Musharraf,4/1/2002,2500.0,0.0,Yes,Yes,Minister Petroleum & Natural\nResources,None
81,Jewellery box,Jewellery/Accessories,Military,4/9/2002,400.0,0.0,Yes,Yes,L/NK M. Aslam,None
82,Jewellery box,Jewellery/Accessories,Military,4/9/2002,400.0,0.0,Yes,Yes,L/NK M. Yousaf,None
83,Jewellery box,Jewellery/Accessories,Military,4/9/2002,400.0,0.0,Yes,Yes,L/ NK Daud Rehman,None
84,Jewellery box,Jewellery/Accessories,Military,4/9/2002,400.0,0.0,Yes,Yes,L/NK Muhammad Asghar,None
88,One Blue Onyx\nBowl,Decoration Pieces,Gen. Musharraf,4/10/2002,500.0,0.0,Retained.,Retained.,Minister for Information & Media\nDevelopment,None


Replacing the null values for the Office with the Recipient name

In [11]:
tosha_df['Office']=tosha_df['Office'].fillna(tosha_df['Recipient'])
#tosha_df.count()

Removing '\n' in columns of Recipient,Office and Detail of Gifts and removing any white spaces.

In [12]:
tosha_df['Office']=tosha_df['Office'].str.replace('\n', ' ')
tosha_df['Recipient']=tosha_df['Recipient'].str.replace('\n',' ')
tosha_df["Detail of Gifts"]=tosha_df['Detail of Gifts'].str.replace('\n', ' ')
tosha_df["Detail of Gifts"]=tosha_df['Detail of Gifts'].str.strip()
tosha_df['Recipient']=tosha_df['Recipient'].str.strip()
tosha_df['Office']=tosha_df['Office'].str.strip()
# replace any '\n' with ' ' with regular expressions
tosha_df=tosha_df.replace('\n',' ',regex=True)

tosha_df.head(10)

,Detail of Gifts,Item Category,Affiliation,Date,Assessed Value,Retention Cost,Retained,Remarks,Recipient,Office
0,Two small silver Bowls,Decoration Pieces,Gen. Musharrafarraf,1/8/2002,5000.0,0.0,Yes,Yes,Mr. Abdul Sattar,Minister for Foreign Affairs
1,One Pilot Pen with ink,Pen,Gen. Musharraf,1/8/2002,3500.0,0.0,Yes,Yes,Mr. Abdul Sattar,Minister for Foreign Affairs
2,One silver vase,Decoration Pieces,Gen. Musharraf,1/9/2002,7500.0,0.0,Yes,Yes,Mr. Shaukat Aziz,Minister for Finance
3,One (decoration) folding Screen,Decoration Pieces,Gen. Musharraf,1/15/2002,2200.0,0.0,Yes,Yes,Mr. Abdul Sattar,Minister for Foreign Affairs
4,One Decoration Piece,Decoration Pieces,Gen. Musharraf,1/22/2002,2000.0,0.0,Yes,Yes,Mr. Abdul Sattar,Minister for Foreign Affairs
5,One gown,Clothes,Gen. Musharraf,1/26/2002,2500.0,0.0,Yes,Yes,Mr. Ahmed Waqar,Secretary Privatization Commission
6,One Coffee Set (07 pcs),Crockery,Gen. Musharraf,1/28/2002,1800.0,0.0,Yes,Yes,Mr. Abdul Sattar,Minister for Foreign Affairs
7,Art glass decoration piece,Decoration Pieces,Gen. Musharraf,2/1/2002,1200.0,0.0,Yes,Yes,Mr. Shaukat Aziz,Minister for Finance
8,One leather black colour wallet,Decoration Pieces,Gen. Musharraf,2/4/2002,150.0,0.0,Auctioned,Auctioned for Rs.500/-,Dr. Tariq Hassan,Advisor to Finance Minister
9,One Pen Stand fitted with clock,Pen,Gen. Musharraf,2/6/2002,550.0,0.0,Auctioned,Auctioned for Rs.305/-,Mr. Altaf M. Saleem,Minster for Privatization


Inconsistencies in values like 'Mr.  Asif  Ali  Zardari', 'President of Pakistan', 'Prime Minister of Pakistan'

In [13]:
print('names:\n',tosha_df["Recipient"].value_counts())
print('Office:\n',tosha_df["Office"].value_counts())
print('Affiliation:\n',tosha_df["Affiliation"].value_counts())

names:
 Mr.  Shaukat  Aziz                     774
Mr.  Asif  Ali  Zardari                167
Mr.  Khurshid  M.  Kasuri              103
Mr.  Imran  Khan                        82
Mir  Zafarullah  Khan Jamali            71
                                      ... 
Mr.  Munir  Khan Orakzai                 1
Mr  Asif  Ali  Zardari                   1
Mr.  Zafar  Mehmood                      1
Ms.  Nuzhat  Sadiq                       1
Military  Secretary  to  the  Prime      1
Name: Recipient, Length: 1589, dtype: int64
Office:
 Prime Minister of Pakistan                          854
President of Pakistan                               273
Prime  Minister  of Pakistan                        119
Foreign Minister                                     88
President  of Pakistan                               74
                                                   ... 
Former Prime Minister of Pakistan                     1
Advisor  to  Prime                                    1
Security Official 

In [14]:
tosha_df=tosha_df.replace('  ',' ',regex=True)
tosha_df=tosha_df.replace('Mr ','Mr.')
print('names:\n',tosha_df["Recipient"].value_counts())
print('Office:\n',tosha_df["Office"].value_counts())
print('Affiliation:\n',tosha_df["Affiliation"].value_counts())
print('Remarks:\n',tosha_df['Remarks'].value_counts())
print('Retained:\n',tosha_df['Retained'].value_counts())
print('Catagories:\n',tosha_df['Item Category'].value_counts())

names:
 Mr. Shaukat Aziz                   815
Mr. Asif Ali Zardari               170
Mr. Khurshid M. Kasuri             104
Dr. Arif Alvi                       99
Mr. Imran Khan                      85
                                  ... 
Sep. M. Habib Sultan                 1
Hav. Sabz Ali Khan                   1
Mr. Amjad Mehmood                    1
Mr. Sohail Ali Khan                  1
Military Secretary to the Prime      1
Name: Recipient, Length: 1429, dtype: int64
Office:
 Prime Minister of Pakistan           992
President of Pakistan                347
Foreign Minister                      88
Finance Minister                      73
Chairman Senate                       68
                                    ... 
Security Official                      1
Advisor to Prime                       1
Former Prime Minister of Pakistan      1
DMS (P), President Sectt.              1
Deputy Secretary (CF-I)                1
Name: Office, Length: 1303, dtype: int64
Affiliation:
 Othe

In [15]:
tosha_df.to_csv('cleaned-tosha-df-v1.csv',index=False)